# Turtlebot Teleop Demo

Make sure there is a robot published, you can launch turtlebot3 waffle with the following commands:

`export TURTLEBOT3_MODEL=waffle`

`roslaunch turtlebot3_fake turtlebot3_fake.launch`

In [ ]:
import numpy as np
import bqplot as bq
import ipywidgets as wd
from ipywidgets import FloatSlider, Layout, Button, Box, FloatText, Label

In [ ]:
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
import rospy

rospy.init_node('commander')

In [ ]:
import jupyros

In [ ]:
# Sliders to control velocity
linear_slider = FloatSlider(min = -1, max = 1, step = 0.05)
angular_slider = FloatSlider(min = -1, max = 1, step = 0.1)

pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
rospy.loginfo("Publisher connected /cmd_vel")

In [ ]:
%%thread_cell

rate = rospy.Rate(5)
while True:
    msg = Twist()
    msg.linear.x = linear_slider.value
    msg.angular.z = angular_slider.value
    pub.publish(msg)
    rate.sleep()

In [ ]:
# Control form
controls = [ Box([Label(value="Linear"), linear_slider]), Box([Label(value="Angular"), angular_slider])]

form = Box(controls, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    padding='10px'
))

#form

In [ ]:
# Live ploting of velocity or position
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

ax_x = bq.Axis(label='X', scale=x_sc, grid_lines='solid')
ax_y = bq.Axis(label='Y', scale=y_sc, orientation='vertical', grid_lines='solid')

lines = bq.Lines(x=np.array([]), y=np.array([]), scales={'x': x_sc, 'y': y_sc})
fig = bq.Figure(axes=[ax_x, ax_y], marks=[lines], labels=["linear","angular"], display_legend=True, title="Velocity")
data = []

def cb(msg, data=data):
    data.append([msg.pose.pose.position.x, msg.pose.pose.position.y])
    data = data[-100:]
    ndat = np.asarray(data).T
    
    if lines:
        lines.y = ndat
        lines.x = np.arange(len(data))

def cb_twist(msg, data=data):
    data.append([msg.twist.twist.linear.x, msg.twist.twist.angular.z])
    data = data[-100:]
    ndat = np.asarray(data).T
    
    if lines:
        lines.y = ndat
        lines.x = np.arange(len(data))

rospy.Subscriber('/odom', Odometry, cb_twist)
rospy.loginfo("Subscribed to /odom")
#fig

In [ ]:
form

In [ ]:
fig